In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
sba_loans = pd.read_csv('selected_sba_loans_data.csv')
sba_loans

,ChgOffPrinGr,Term,UrbanRural,RevLineCr,LowDoc,BankState,State,NAICS,MIS_Status
0,-0.207258,-0.339513,0,0.0,1.0,38,15,831,0.0
1,-0.207258,-0.643861,0,0.0,1.0,18,15,1233,0.0
2,-0.207258,0.877876,0,0.0,0.0,18,15,1157,0.0
3,-0.207258,-0.643861,0,0.0,1.0,39,36,0,0.0
4,-0.207258,1.638745,0,0.0,0.0,11,9,0,0.0
...,...,...,...,...,...,...,...,...,...
899159,-0.207258,-0.643861,0,0.0,0.0,17,35,831,0.0
899160,-0.207258,-0.643861,0,1.0,0.0,17,35,832,0.0
899161,-0.207258,-0.035166,0,0.0,0.0,5,4,436,0.0
899162,0.504659,-0.643861,0,0.0,1.0,14,11,0,1.0


In [3]:
# Drop rows with any NaN values
sba_loans = sba_loans.dropna()

In [4]:
sba_loans.info()

<class 'pandas.core.frame.DataFrame'>
Index: 872919 entries, 0 to 899163
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ChgOffPrinGr  872919 non-null  float64
 1   Term          872919 non-null  float64
 2   UrbanRural    872919 non-null  int64  
 3   RevLineCr     872919 non-null  float64
 4   LowDoc        872919 non-null  float64
 5   BankState     872919 non-null  int64  
 6   State         872919 non-null  int64  
 7   NAICS         872919 non-null  int64  
 8   MIS_Status    872919 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 66.6 MB


In [5]:
# List of columns to convert to int64
columns_to_convert = ['UrbanRural', 'RevLineCr', 'LowDoc', 'BankState', 'State', 'NAICS', 'MIS_Status']

# Convert each specified column to int64
for column in columns_to_convert:
    sba_loans[column] = sba_loans[column].astype('int64', errors='ignore')  # errors='ignore' will skip incompatible columns

/var/folders/pk/ntjyb03d4djbyplk8mmb55jc0000gn/T/ipykernel_69268/2714612023.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sba_loans[column] = sba_loans[column].astype('int64', errors='ignore')  # errors='ignore' will skip incompatible columns
/var/folders/pk/ntjyb03d4djbyplk8mmb55jc0000gn/T/ipykernel_69268/2714612023.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sba_loans[column] = sba_loans[column].astype('int64', errors='ignore')  # errors='ignore' will skip incompatible columns
/var/folders/p

In [6]:
from sklearn.model_selection import train_test_split

# Define the target variable and feature set
X = sba_loans.drop(columns=['MIS_Status'])
y = sba_loans[['MIS_Status']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.linear_model import LogisticRegression

# Train the logistic regression model on the training data
clf_logistic = LogisticRegression(solver='lbfgs', max_iter=1000).fit(X_train, np.ravel(y_train))

# Predict the probability of default on the entire test set
preds = clf_logistic.predict_proba(X_test)

# Create DataFrames of all predictions (probability of default) and true labels
preds_df = pd.DataFrame(preds[:, 1], columns=['prob_default'])
true_df = y_test.reset_index(drop=True)

# Combine the true labels and predicted probabilities for the entire dataset
result_df = pd.concat([true_df, preds_df], axis=1)

# Format the 'prob_default' column as percentages
result_df['prob_default'] = result_df['prob_default'].apply(lambda x: "{:.1%}".format(x))

In [8]:
result_df

,MIS_Status,prob_default
0,0,0.2%
1,0,10.1%
2,0,0.1%
3,0,0.1%
4,0,0.1%
...,...,...
174579,0,3.1%
174580,0,0.0%
174581,0,0.0%
174582,0,0.9%


In [10]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize and train the k-NN model (choose k=5 as an example, but you can adjust it)
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, np.ravel(y_train))

# Predict the probability of default on the entire test set
preds = knn_model.predict_proba(X_test)

# Create DataFrames of all predictions (probability of default) and true labels
preds_df = pd.DataFrame(preds[:, 1], columns=['prob_default'])
true_df = y_test.reset_index(drop=True)

# Combine the true labels and predicted probabilities for the entire dataset
result_df = pd.concat([true_df, preds_df], axis=1)

# Format the 'prob_default' column as percentages
result_df['prob_default'] = result_df['prob_default'].apply(lambda x: "{:.1%}".format(x))

In [13]:
result_df

,MIS_Status,prob_default
0,0,0.0%
1,0,0.0%
2,0,0.0%
3,0,0.0%
4,0,0.0%
...,...,...
174579,0,0.0%
174580,0,0.0%
174581,0,0.0%
174582,0,0.0%
